In [1]:
import numpy as np
from pyquil.quil import Program
from pyquil.gates import *
from pyquil.quilbase import *

from unitary_generator import apply_gate, permutation_arbitrary, lifted_gate
from gates import gate_matrix, utility_gates

In [2]:
import pyquil.quil as pq 
import api as api
from pyquil.gates import *

# open a 'connection' to the reference-qvm
qvm = api.Connection()

## Basic tests

In [3]:
p = Program().inst(H(0), H(1))
p.measure(0, [0])
m_obj = p.synthesize()[1]
wf, data = qvm.wavefunction(p)
print wf.amplitudes
print data
print wf

[[ 0.00000000+0.j]
 [ 0.70710678+0.j]
 [ 0.00000000+0.j]
 [ 0.70710678+0.j]]
[array([ True], dtype=bool)]
(0.7071067812+0j)|01> + (0.7071067812+0j)|11>


## Single-shot Grover's

In [4]:
data = [0, 0, 0, 1]

N = len(data)
n = int(np.log2(N))
assert(N > 0)

# setup program
p = Program()

# run program on qvm
qvm = api.Connection()

# initialize n qubits to the mixed state
for i in range(n):
    p.inst(H(i))
    
# oracle operator
outer_w = np.diag(data)
Uw = np.identity(N) - 2 * outer_w
p.defgate("G-ORAC", Uw)
inst_orac = tuple(["G-ORAC"] + range(n))

# diffusion operator
Us = (2. / N) * np.ones(N * N).reshape((N, N)) - np.identity(N)
p.defgate("G-DIFF", Us)
inst_diff = tuple(["G-DIFF"] + range(n))

# single-shot Grover iteration
p.inst(inst_orac, inst_diff)

# measurement
for i in range(n):
    p.measure(i, [i])

In [5]:
print p

DEFGATE G-ORAC:
    1.0, 0.0, 0.0, 0.0
    0.0, 1.0, 0.0, 0.0
    0.0, 0.0, 1.0, 0.0
    0.0, 0.0, 0.0, -1.0

DEFGATE G-DIFF:
    -0.5, 0.5, 0.5, 0.5
    0.5, -0.5, 0.5, 0.5
    0.5, 0.5, -0.5, 0.5
    0.5, 0.5, 0.5, -0.5

H 0
H 1
G-ORAC 0 1
G-DIFF 0 1
MEASURE 0 [0]
MEASURE 1 [1]



In [6]:
# results are returned in the measured classical registers
results = qvm.run(p, range(n))

In [7]:
ret = int(''.join([str(int(e)) for e in results[0]]), 2)
assert(data[ret] == 1)

In [8]:
results

[[True, True]]

## Maximum qubit test

In [9]:
# setup program
p = Program()

# run program on qvm
qvm = api.Connection()

n = 20
# initialize n qubits to the mixed state
for i in range(n):
    p.inst(H(i))

In [10]:
mask = None
print list(np.array([])[mask])

[array([], dtype=float64)]


In [11]:
print p

# results are returned in the measured classical registers
results = qvm.wavefunction(p)

H 0
H 1
H 2
H 3
H 4
H 5
H 6
H 7
H 8
H 9
H 10
H 11
H 12
H 13
H 14
H 15
H 16
H 17
H 18
H 19



## Grove algorithm testing

In [12]:
import numpy as np
from grove.pyqaoa.maxcut_qaoa import maxcut_qaoa
qvm_connection = api.Connection()

In [13]:
square_ring = [(0,1),(1,2),(2,3),(3,0)]
steps = 2; n_qubits = 4
p = steps
betas = np.random.uniform(0, np.pi, p); gammas = np.random.uniform(0, 2*np.pi, p)
inst = maxcut_qaoa(square_ring, steps=steps)
inst.get_angles()

                     models will be ineffective
	Parameters: [ 0.69015608  0.32325579  5.98663447  1.4742472 ] 
	E => -1.43441749975
	Parameters: [ 0.69015608  0.32325579  5.98663447  1.4742472 ] 
	E => -1.48354226052
	Parameters: [ 0.748388    0.32022527  6.18262548  1.39132079] 
	E => -2.13090508162
	Parameters: [ 0.69716548  0.31795238  6.54342711  1.32912599] 
	E => -3.00961955614
	Parameters: [ 0.69716548  0.31795238  6.54342711  1.32912599] 
	E => -2.69806660226
	Parameters: [ 0.70855575  0.32751747  7.05623314  1.3328692 ] 
	E => -3.59101186044
	Parameters: [ 0.70855575  0.32751747  7.05623314  1.3328692 ] 
	E => -3.48271091508
	Parameters: [ 0.70855575  0.32751747  7.05623314  1.3328692 ] 
	E => -3.43970075514
	Parameters: [ 0.70855575  0.32751747  7.05623314  1.3328692 ] 
	E => -3.24784849774
	Parameters: [ 0.70855575  0.32751747  7.05623314  1.3328692 ] 
	E => -3.56520932195
	Parameters: [ 0.70855575  0.32751747  7.05623314  1.3328692 ] 
	E => -3.50604656507
	Parameters: [ 0.

(array([ 0.55843768,  0.45382061]), array([ 7.18760282,  1.12257026]))

In [14]:
t = np.hstack((inst.betas, inst.gammas))
param_prog = inst.get_parameterized_program()
prog = param_prog(t)
wf, _ = qvm_connection.wavefunction(prog)
wf = wf.amplitudes

In [15]:
for state_index in range(2**inst.n_qubits):
    print inst.states[state_index], np.conj(wf[state_index])*wf[state_index]

0000 [ 0.27144661+0.j]
0001 [ 0.01612827+0.j]
0010 [ 0.01612827+0.j]
0011 [ 0.06470399+0.j]
0100 [ 0.01612827+0.j]
0101 [ 0.03463232+0.j]
0110 [ 0.06470399+0.j]
0111 [ 0.01612827+0.j]
1000 [ 0.01612827+0.j]
1001 [ 0.06470399+0.j]
1010 [ 0.03463232+0.j]
1011 [ 0.01612827+0.j]
1100 [ 0.06470399+0.j]
1101 [ 0.01612827+0.j]
1110 [ 0.01612827+0.j]
1111 [ 0.27144661+0.j]


## Labels, control flow

In [4]:
prog = Program(X(0))

prog.inst(HALT)

prog.inst(X(0))

In [6]:
print prog.synthesize()

[<Gate: X 0>, <pyquil.quilbase.Halt object at 0x7f2f70b6d090>, <Gate: X 0>]


## other QVM bugs

In [3]:
p = Program().inst(X(0))
qvm.run_and_measure(p, [0, 1, 2], trials=1)

[[1, 0, 0]]